## Final Project Submission

* John Paul Hernandez Alcala
* Part Time
* Scheduled project review date/time: 4/30/2020 -- 11:30 am CST
* Instructor name: Eli
* Blog post URL:


### Libraries used

In [1]:
#!pip install omdb #if not installed already

import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn
import omdb
import itertools as it


## Movie Budget Data

First, we will read our comma-separated movie budget data into a Pandas dataframe for easier manipulation.

In [2]:
df_original= pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
df_original.head() #See if the data is organized correctly

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [3]:
df_original.info() #Quick look at the number of data points, number of columns, and other details

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


From the above info about the dataframe, we can see that production_budget, domestic_gross, and worldwide_gross are currently object dtypes. These need to be changed to int64 dtype, so we can do calcuations later. Before messing with the dataframe, we will make a copy of the original.


### Data Cleaning and Wrangling

In [4]:
df = df_original.copy()
df.head() #Always display to make sure we did not screw it up

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


First, we need to remove the "$" and "," from the data. The two methods which I am aware of are below [1]:

In [5]:
#Using normal panadas series.str.replace function
df.production_budget = df.production_budget.str.replace("$", "") #look for this symbol and replace with with nothing
df.production_budget = df.production_budget.str.replace(",", "")

#Using pandas series.replace function
df.domestic_gross.replace({"\$":"", ",":""}, regex=True, inplace=True) #Same as above except we look for two symbols and keep changes
df.worldwide_gross.replace({"\$":"", ",":""}, regex=True, inplace=True)

df.head()#yep, display again

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


Now we can convert to these to int64

In [6]:
df = df.astype({'production_budget':'int64', 'domestic_gross':'int64', 'worldwide_gross':'int64'})

If there are any "NaN" or values that did not have "$" and/or "," they should appear as a class other than <class 'int'>

In [7]:
print("{}\n{}\n{}".format(df.production_budget.apply(type).value_counts(), df.domestic_gross.apply(type).value_counts(),
                          df.worldwide_gross.apply(type).value_counts()))

<class 'int'>    5782
Name: production_budget, dtype: int64
<class 'int'>    5782
Name: domestic_gross, dtype: int64
<class 'int'>    5782
Name: worldwide_gross, dtype: int64


We check for extreme values in these columns.

In [8]:
print(df.production_budget.agg(['min', 'max', 'mean', 'std']))#std to give us an idea of anything is moving towards a certain way
print(df.domestic_gross.agg(['min', 'max', 'mean', 'std']))
print(df.worldwide_gross.agg(['min', 'max', 'mean', 'std']))

min     1.100000e+03
max     4.250000e+08
mean    3.158776e+07
std     4.181208e+07
Name: production_budget, dtype: float64
min     0.000000e+00
max     9.366622e+08
mean    4.187333e+07
std     6.824060e+07
Name: domestic_gross, dtype: float64
min     0.000000e+00
max     2.776345e+09
mean    9.148746e+07
std     1.747200e+08
Name: worldwide_gross, dtype: float64



We will get rid of data that does not have domestic gross since the US is a top box office territory according to a report by Amy Watson [2]

In [9]:
df = df[df.domestic_gross != 0]

In [10]:
df.info() #Notice how we don't have 5782 values anymore

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5234 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5234 non-null   int64 
 1   release_date       5234 non-null   object
 2   movie              5234 non-null   object
 3   production_budget  5234 non-null   int64 
 4   domestic_gross     5234 non-null   int64 
 5   worldwide_gross    5234 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 286.2+ KB


In [11]:
print(df.production_budget.agg(['min', 'max', 'mean', 'std']))#std to give us an idea of anything is moving towards a certain way
print(df.domestic_gross.agg(['min', 'max', 'mean', 'std']))
print(df.worldwide_gross.agg(['min', 'max', 'mean', 'std']))

min     1.100000e+03
max     4.250000e+08
mean    3.403348e+07
std     4.296048e+07
Name: production_budget, dtype: float64
min     3.880000e+02
max     9.366622e+08
mean    4.625747e+07
std     7.029651e+07
Name: domestic_gross, dtype: float64
min     4.010000e+02
max     2.776345e+09
mean    1.007615e+08
std     1.811226e+08
Name: worldwide_gross, dtype: float64


From above we observe that ~90% of the original data has domestic box office

We shoudld sort the name of the movies just to see if we have weird characters.

In [12]:
df.sort_values(['movie'], axis=0, ascending=False).head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
2701,2,"Mar 7, 2008",é·æ±ä¸è (CJ7),20000000,206678,47300771
514,15,"Jan 20, 2017",xXx: Return of Xander Cage,85000000,44898413,345033359
697,98,"Aug 9, 2002",xXx,70000000,141930000,267200000
1958,59,"Sep 15, 2017",mother!,30000000,17800004,42531076
2482,83,"Apr 23, 1999",eXistenZ,20700000,2840417,2840417


And it looks like we in fact do have weird characters in some of our movie titles

We will use regression to find any characters that are not found on a typical keyboard or hex 0x20 (i.e. space) - 0x7E (~) from ASCII [3]

In [13]:
movieCol = df.movie.map(lambda x: int(bool(re.search(r'[^\x20-\x7E]', x)))) # ^ stands for not, [] is the list of char, and - is the range


"moviecol" is the movie series from df that has weird characters in it. We will now see which movies from the df have weird characters in it below:

In [14]:
dfweird = df[movieCol == 1] # "==" means we want weird data

In [15]:
dfweird.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
7,8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,300000000,309420425,963420425
27,28,"Jul 7, 2006",Pirates of the Caribbean: Dead Manâs Chest,225000000,423315812,1066215812
70,71,"Nov 6, 2009",Disneyâs A Christmas Carol,190000000,137855863,315709697
91,92,"Jul 27, 2018",Mission: ImpossibleâFallout,178000000,220159104,787456552
167,68,"Jul 31, 2015",Mission: ImpossibleâRogue Nation,150000000,195042377,688858992


We will take out all these data points.

In [16]:
dfv1 = df[movieCol != 1] # "!=" means we do not want weird data
dfv1.info() #We lost a little more data points from the 5234

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5126 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5126 non-null   int64 
 1   release_date       5126 non-null   object
 2   movie              5126 non-null   object
 3   production_budget  5126 non-null   int64 
 4   domestic_gross     5126 non-null   int64 
 5   worldwide_gross    5126 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 280.3+ KB


In [17]:
dfv1

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5775,76,"May 26, 2006",Cavite,7000,70071,71644
5776,77,"Dec 31, 2004",The Mongol King,7000,900,900
5778,79,"Apr 2, 1999",Following,6000,48482,240495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338


Now let us check if there are any duplicate titles in our dfv1.

In [18]:
dfv1
mlist = list(dfv1.movie) #get list of movie titles
mlist2 = set(mlist) #get second list of unique movie titles
print('Number of Nonunique Titles: ', len(mlist) - len(mlist2))
list3 = []
for m in mlist2: #goes through the unique list of movies
    if mlist.count(m) > 1: #we check to see which movies appears more than once in the original list of movies
        list3.append(m) #add the movie that appears more than once to list3

Number of Nonunique Titles:  74


Let's check if it is actually a duplicate

In [19]:
dfv1[dfv1.movie == 'Crash'] 

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
3669,70,"Oct 4, 1996",Crash,10000000,3357324,3357324
3978,79,"May 6, 2005",Crash,7303082,55334418,101173038


### Store dfmoviebudget Variable

From here, we can move on to the anaylsis of this dataset in [MovieBudgetAnalysis.ipynb](./MovieBudgetAnalysis.ipynb), or extract data from the OMDb API, [OMDb_Data.ipynb](./OMDb_Data.ipynb). Either way, we will store the below variable for later usage.

In [20]:
dfmoviebudget = dfv1
%store dfmoviebudget

Stored 'dfmoviebudget' (DataFrame)


### Resources used for development:
1. https://pbpython.com/currency-cleanup.html
2. https://www.statista.com/statistics/252730/leading-film-markets-worldwide--gross-box-office-revenue/
3. https://stackoverflow.com/questions/45147479/python-regex-for-removing-strange-characters
4. https://www.researchgate.net/publication/313455341_Predicting_Movie_Box_Office_Profitability_A_Neural_Network_Approach
5. http://www.omdbapi.com/

